## 라이브러리 로드

In [1]:
# 라이브러리 로드
# requests는 작은 웹브라우저로 웹사이트 내용을 가져옵니다.
import requests
# BeautifulSoup 을 통해 읽어 온 웹페이지를 파싱합니다.
from bs4 import BeautifulSoup as bs
# 크롤링 후 결과를 데이터프레임 형태로 보기 위해 불러옵니다.
import pandas as pd

## 서울시 코로나19 발생동향
* [코로나19](http://www.seoul.go.kr/coronaV/coronaStatus.do)

## requests 로 html 문서 받아오기

In [2]:
# 크롤링 할 사이트
base_url = "http://www.seoul.go.kr/coronaV/coronaStatus.do"
response = requests.get( base_url )
response

<Response [200]>

In [3]:
if response.status_code == 200:
    soup = bs(response.text, 'html.parser')

str(soup)[:1000]

'\n<!DOCTYPE HTML>\n\n<html lang="ko">\n<head>\n<meta charset="utf-8"/>\n<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>\n<meta content="width=device-width, initial-scale=1.0" name="viewport">\n<title>서울시 코로나19</title>\n<meta content="서울시 코로나19 통합정보 사이트로 안전·방역(발생동향, 클린존, 선별진료소, 해외입국자 안내, 일일브리핑, 홍보물&amp;공적 공급마스크, 보도자료, 일일 소식지&amp;대응일지, 신고(응답소)), 생활정보(주요뉴스, 생활경제 지원, 온라인전시&amp;공연, 온라인도서관, 온라인체육관, 시민생활수기, 언론사 팩트체크, 심리지원), 시민 참여(잠시멈춤 캠페인, 온-서울캠페인, 시민제안)등의 정보를 제공하고 있습니다." name="description"/>\n<meta content="website" property="og:type"/>\n<meta content="서울시 코로나19 통합사이트" property="og:title"/>\n<meta content="서울시 코로나19 통합정보 사이트로 안전·방역(발생동향, 클린존, 선별진료소, 해외입국자 안내, 일일브리핑, 홍보물&amp;공적 공급마스크, 보도자료, 일일 소식지&amp;대응일지, 신고(응답소)), 생활정보(주요뉴스, 생활경제 지원, 온라인전시&amp;공연, 온라인도서관, 온라인체육관, 시민생활수기, 언론사 팩트체크, 심리지원), 시민 참여(잠시멈춤 캠페인, 온-서울캠페인, 시민제안)등의 정보를 제공하고 있습니다." property="og:description"/>\n<meta content="https://www.seoul.go.kr/coronaV/coronaStatus.do" property="og:url"/>\n<meta content="http://www.seoul.go

## 서울 확진자 현황

In [4]:
#move-cont1 > div:nth-child(2) > div
#DataTables_Table_0 > thead > tr
# cont_page = soup.select("#move-cont1 > div.status-confirm > div.cont-page-wrap > div.cont-page")
#DataTables_Table_0
# <table class="tstyle05 tstyleP status-datatable datatable-multi-row">
cont_page = soup.select(".datatable-multi-row")[0]
print(len(cont_page))
cont_page

9


<table class="tstyle05 tstyleP status-datatable datatable-multi-row">
<caption>서울 확진자 현황</caption>
<colgroup>
<col style="width: 10%;"/>
<col style="width: 10%;"/>
<col style="width: 10%;"/>
<col style="width: 10%;"/>
<col style="width: 20%;"/>
<col style="width: 20%;"/>
<col style="width: 20%;"/>
</colgroup>
<thead>
<tr>
<th scope="col">연번</th>
<th scope="col">환자</th>
<th scope="col">확진일</th>
<th scope="col">거주지</th>
<th scope="col">여행력</th>
<th scope="col">접촉력</th>
<th scope="col">조치사항</th>
</tr>
</thead>
<tbody>
<tr>
<th scope="row"><p class="corona19_no">1113</p></th>
<td data-tit="환자 번호" scope="col">12078</td>
<td data-tit="확진일">6.13.</td>
<td data-tit="거주지">영등포구</td>
<td data-tit="여행력">-</td>
<td data-tit="접촉력">확인 중</td>
<td data-tit="조치사항">서남병원</td>
</tr>
<tr>
<th scope="row"><p class="corona19_no">1112</p></th>
<td data-tit="환자 번호" scope="col">12079</td>
<td data-tit="확진일">6.13.</td>
<td data-tit="거주지">서초구</td>
<td data-tit="여행력">-</td>
<td data-tit="접촉력">리치웨이 관련</td>
<td data-

In [5]:
trs_ex = cont_page.select("tr")
print(len(trs_ex))
trs_ex[-1]

1114


<tr>
<th scope="row"><p class="corona19_no">1</p></th>
<td data-tit="환자 번호" scope="col">2</td>
<td data-tit="확진일">1.24.</td>
<td data-tit="거주지">강서구</td>
<td data-tit="여행력">중국 우한시</td>
<td data-tit="접촉력">해외 접촉</td>
<td data-tit="조치사항">국립중앙의료원(<b class="status1">퇴원</b>)</td>
</tr>

## 행 데이터 리스트로 만들기

In [6]:
def parse_tr(trs):
    table = []
    for tr in trs:
        tds = tr.select("td")
        if len(tds) > 0 :
            row = []
            # 연번
            number = tr.select("th")[0].get_text()
            row.append(number)
            # 연번 외 데이터
            for td in tds:
                val = td.get_text()
                row.append(val)
            table.append(row)
    return table

parse_tr(trs_ex[:3])

[['1113', '12078', '6.13.', '영등포구', '-', '확인 중', '서남병원'],
 ['1112', '12079', '6.13.', '서초구', '-', '리치웨이 관련', '보라매병원']]

## 페이지마다 가져오기

In [7]:
#move-cont1 > div:nth-child(2) > table
#move-cont1 > div:nth-child(2) > table.tstyle05.tstyleP > thead > tr > th:nth-child(1)
#cont-page2
# table_result = []
# for page in cont_page:
#     trs = page.select("tr")
#     result = parse_tr(trs)
#     table_result.extend(result)

# table_result[:5]

In [8]:
table_result = parse_tr(trs_ex)
table_result[-1]

['1', '2', '1.24.', '강서구', '중국 우한시', '해외 접촉', '국립중앙의료원(퇴원)']

In [9]:
table_result[0]

['1113', '12078', '6.13.', '영등포구', '-', '확인 중', '서남병원']

## 컬럼명 만들기

In [10]:
# 컬럼명 만들기
cols = trs_ex[0].select("th")
cols

[<th scope="col">연번</th>,
 <th scope="col">환자</th>,
 <th scope="col">확진일</th>,
 <th scope="col">거주지</th>,
 <th scope="col">여행력</th>,
 <th scope="col">접촉력</th>,
 <th scope="col">조치사항</th>]

In [11]:
col_name = []
for col in cols:
    col_name.append(col.get_text())
col_name

['연번', '환자', '확진일', '거주지', '여행력', '접촉력', '조치사항']

## 데이터프레임으로 만들기

In [12]:
df = pd.DataFrame(table_result, columns=col_name)
print(df.shape)
df.head()

(1113, 7)


,연번,환자,확진일,거주지,여행력,접촉력,조치사항
0,1113,12078,6.13.,영등포구,-,확인 중,서남병원
1,1112,12079,6.13.,서초구,-,리치웨이 관련,보라매병원
2,1111,12068,6.13.,주소불명,-,확인 중,보라매병원
3,1110,12067,6.13.,관악구,-,리치웨이 관련,생활치료센터
4,1109,12064,6.13.,중랑구,-,리치웨이 관련,서울의료원


## CSV 파일로 저장하기

In [13]:
import datetime

today = datetime.datetime.today()
now = today.strftime('%Y-%m-%d')
now

'2020-06-14'

In [14]:
df.to_csv(f"covid-19-seoul-{now}.csv", index=False)

## CSV 파일 읽어오기

In [15]:
pd.read_csv(f"covid-19-seoul-{now}.csv").head(10)

,연번,환자,확진일,거주지,여행력,접촉력,조치사항
0,1113,12078,6.13.,영등포구,-,확인 중,서남병원
1,1112,12079,6.13.,서초구,-,리치웨이 관련,보라매병원
2,1111,12068,6.13.,주소불명,-,확인 중,보라매병원
3,1110,12067,6.13.,관악구,-,리치웨이 관련,생활치료센터
4,1109,12064,6.13.,중랑구,-,리치웨이 관련,서울의료원
5,1108,12059,6.13.,금천구,-,리치웨이 관련,보라매병원
6,1107,12069,6.13.,영등포구,-,연아나뉴스클래스 관련,서남병원
7,1106,12062,6.13.,강서구,-,양천구 운동시설 관련,보라매병원
8,1105,12063,6.13.,강서구,-,한국대학생선교회 관련,생활치료센터
9,1104,12058,6.13.,종로구,-,확인 중,은평성모병원


In [16]:
df["접촉력"].value_counts()

해외 접촉 추정      263
이태원 클럽 관련     139
리치웨이 관련        86
구로구 콜센터 관련     60
확인 중           60
             ... 
#3 접촉           1
쇼핑몰 관련          1
#794 접촉자        1
#10115 접촉       1
#136 접촉         1
Name: 접촉력, Length: 110, dtype: int64

In [17]:
df["조치사항"].value_counts()

서울의료원(퇴원)      212
서울의료원          141
보라매병원(퇴원)      138
서남병원(퇴원)       104
서남병원            99
              ... 
서울의료원(사망)        1
삼성서울병원(퇴원)       1
국군수도병원           1
중앙대학병원(퇴원)       1
경희대학교병원(퇴원)      1
Name: 조치사항, Length: 65, dtype: int64